<h1>Process the GHG data</h1>

In [1]:
import pandas as pd
import numpy as np

In [2]:
ghg_raw = pd.read_json('ghg.json')

In [3]:
ghg_raw.head()

,Report_Year,Energy,Waste,IndProc_and_ProdUse,AgrFor_and_Other,Total
0,1990,"{'EG_and_Tg_Prod': 22900, 'Transport': 6160, '...",1550,215,139,35800
1,1991,"{'EG_and_Tg_Prod': 25600, 'Transport': 6720, '...",1600,638,121,39200
2,1992,"{'EG_and_Tg_Prod': 29200, 'Transport': 7110, '...",1660,651,99,43500
3,1993,"{'EG_and_Tg_Prod': 29700, 'Transport': 7210, '...",1750,724,86,43800
4,1994,"{'EG_and_Tg_Prod': 21900, 'Transport': 7520, '...",1770,830,76,36400


"Energy" is a nested field, gonna break it down:

In [4]:
EG_and_Tg_Prod = []
Transport = []
Other = []
for energy in ghg_raw.Energy:
    EG_and_Tg_Prod.append(energy['EG_and_Tg_Prod'])
    Transport.append(energy['Transport'])
    Other.append(energy['Other'])
ghg = ghg_raw.drop('Energy', axis=1)
ghg['EG_and_Tg_Prod'] = EG_and_Tg_Prod
ghg['Transport'] = Transport
ghg['Other'] = Other

In [5]:
ghg.head()

,Report_Year,Waste,IndProc_and_ProdUse,AgrFor_and_Other,Total,EG_and_Tg_Prod,Transport,Other
0,1990,1550,215,139,35800,22900,6160,4840
1,1991,1600,638,121,39200,25600,6720,4510
2,1992,1660,651,99,43500,29200,7110,4720
3,1993,1750,724,86,43800,29700,7210,4320
4,1994,1770,830,76,36400,21900,7520,4240


Calculate the following data;
1. The emission difference in each year with the mean value across all years (both actual values and SD)
2. The portion each sector contributes to the total emission in each year (in percentage)

In [6]:
temp = ghg.loc[:, ghg.columns != 'Report_Year']
industry_sd_ghg = (temp-temp.mean())/temp.std()
industry_diff_ghg = ghg-ghg.mean()
for industry in industry_sd_ghg.columns:
    ghg[industry + '_SD'] = industry_sd_ghg[industry]
    ghg[industry + '_Diff'] = industry_diff_ghg[industry]
    ghg[industry + '_Portion'] = ghg[industry]/ghg['Total']

In [7]:
ghg.head()

,Report_Year,Waste,IndProc_and_ProdUse,AgrFor_and_Other,Total,EG_and_Tg_Prod,Transport,Other,Waste_SD,Waste_Diff,...,Total_Portion,EG_and_Tg_Prod_SD,EG_and_Tg_Prod_Diff,EG_and_Tg_Prod_Portion,Transport_SD,Transport_Diff,Transport_Portion,Other_SD,Other_Diff,Other_Portion
0,1990,1550,215,139,35800,22900,6160,4840,-0.995564,-509.354839,...,1.0,-0.864119,-3009.677419,0.639665,-3.548353,-1152.903226,0.172067,2.077438,2064.516129,0.135196
1,1991,1600,638,121,39200,25600,6720,4510,-0.897836,-459.354839,...,1.0,-0.088913,-309.677419,0.653061,-1.824810,-592.903226,0.171429,1.745372,1734.516129,0.115051
2,1992,1660,651,99,43500,29200,7110,4720,-0.780562,-399.354839,...,1.0,0.944696,3290.322581,0.671264,-0.624486,-202.903226,0.163448,1.956686,1944.516129,0.108506
3,1993,1750,724,86,43800,29700,7210,4320,-0.604652,-309.354839,...,1.0,1.088252,3790.322581,0.678082,-0.316711,-102.903226,0.164612,1.554183,1544.516129,0.098630
4,1994,1770,830,76,36400,21900,7520,4240,-0.565561,-289.354839,...,1.0,-1.151232,-4009.677419,0.601648,0.637393,207.096774,0.206593,1.473682,1464.516129,0.116484


In [8]:
pd.DataFrame.to_csv(ghg, 'ghg_processed.csv')